# Action-Angle Coordinates Tutorial

In this tutorial, we will use Agama to compute the *action-angle coordinates*. 

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import agama

%matplotlib inline

mpl.rcParams.update({
    'font.size': 20,
    'figure.figsize': (8, 6),
    'figure.facecolor': 'w',
    'axes.linewidth': 2,
    'xtick.direction': 'in',
    'ytick.direction': 'in',
})

## Calculate the potential

In order to calculate the action-angle coordinates, we first need to estimate the potential $U(x)$ of the system.

In [ ]:
# Read in coordinate of particles
i = 0
subhalo_id = all_subhalo_ids[i]
rot = rotation[i]

# outdir = f'potential/subhalo_{subhalo_id}'
# os.makedirs(outdir, exist_ok=True)

subhalo_props = il.groupcat.loadSingle(basePath, 99, subhaloID=subhalo_id)
x_dm = il.snapshot.loadSubhalo(basePath, 99, subhalo_id, 1, fields=['Coordinates', ])
x_dm = (x_dm - subhalo_props['SubhaloPos']) / h
x_dm = x_dm.dot(rot.T)

data = il.snapshot.loadSubhalo(
    basePath, 99, subhalo_id, 0, fields=['Coordinates', 'Masses', 'InternalEnergy', 'ElectronAbundance'])
x_gas = (data['Coordinates'] - subhalo_props['SubhaloPos']) / h
x_gas = x_gas.dot(rot.T)
m_gas = il_to_msun(data['Masses'])
m_gas_agama = msun_to_agama(m_gas)
temp_gas = calc_temperature(data['InternalEnergy'], data['ElectronAbundance']).value

data = il.snapshot.loadSubhalo(basePath, 99, subhalo_id, 4, fields=['Coordinates', 'Masses'])
x_star = (data['Coordinates'] - subhalo_props['SubhaloPos']) / h
x_star = x_star.dot(rot.T)
m_star = il_to_msun(data['Masses'])
m_star_agama = msun_to_agama(m_star)

In [ ]:
gas_bar_select = temp_gas < 1e5

# calculate the bar potential
particles_bar = (
    np.vstack([x_gas[gas_bar_select], x_star]), 
    np.concatenate([m_gas_agama[gas_bar_select], m_star_agama])
)
potential_bar = agama.Potential(
    type='CylSpline', symmetry='Axisymmetric', gridSizeR=25, gridSizeZ=25, mmax=4,
    particles=particles_bar, )

# calculate the halo potential
particles_halo = (
    np.vstack([x_gas[~gas_bar_select], x_dm]), 
    np.concatenate([m_gas_agama[~gas_bar_select], np.ones(len(x_dm)) * m_dm_agama])
)
potential_halo = agama.Potential(
    type='Multipole', symmetry='Axissymmetric', gridSizeR=25, lmax=4, 
    particles=particles_halo)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6), sharey=True, sharex=True)

r_max = 100
X = np.linspace(0.1, r_max, 200)
Y = np.linspace(0.1, r_max, 200)
R = np.sqrt(X**2 + Y**2)

for z in [0.01, 1, 10, 40]:

    Z = np.ones_like(X) * z
    simulated_pos = np.stack([X, Y, Z], axis=1)
    
    # plot halo potential
    pot = potential_halo.potential(simulated_pos)
    axes[0].plot(R, pot, label=f'$z$ = {z} kpc')
    
    # plot bar potential
    pot = potential_bar.potential(simulated_pos)
    axes[1].plot(R, pot, label=f'$z$ = {z} kpc')
    
axes[0].set_ylabel(r'$\Phi(r) \; [\mathrm{kpc}^2 / \mathrm{Gyr}^2]$', fontsize=20)
axes[0].set_xlabel('$r$ [kpc]', fontsize=20)
axes[1].set_xlabel('$r$ [kpc]', fontsize=20)
axes[0].set_title('Halo')
axes[1].set_title('Bar')
axes[0].legend(loc=4)

fig.tight_layout()

## Calculate the action-angle coordinates

In [ ]:
# Read in coordinate of particles
i = 0
subhalo_id = all_subhalo_ids[i]
rot = rotation[i]

subhalo_props = il.groupcat.loadSingle(basePath, 99, subhaloID=subhalo_id)
data = il.snapshot.loadSubhalo(basePath, 99, subhalo_id, 4, fields=['Coordinates', 'Velocities'])
x_star = (data['Coordinates'] - subhalo_props['SubhaloPos']) / h
x_star = x_star.dot(rot.T)
v_star = data['Velocities'] - subhalo_props['SubhaloVel']
v_star = v_star.dot(rot.T)
xv_star = np.hstack([x_star, v_star])

potential_halo = agama.Potential(file=f'data/potential/subhalo_{subhalo_id}/99.dark.axi_4.coef_mul_DR')
potential_bar = agama.Potential(file=f'data/potential/subhalo_{subhalo_id}/99.bar.axi_4.coef_cylsp_DR')
potential_total = agama.Potential(potential_halo, potential_bar)
action_finder = agama.ActionFinder(potential_total)

# calculate action
action_star = action_finder(xv_star)
Jr, Jz, Jphi = action_star.T / 1000

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

hist_range = ((0, 15), (0, 15), (-15, 15))
norm = mpl.colors.LogNorm(vmin=1, vmax=1000)
bins = 400

select = ~(np.isnan(Jr) | (np.isnan(Jz)) | np.isnan(Jphi))
axes[0].hist2d(
    Jphi[select], Jr[select], bins=(bins, bins), 
    range=(hist_range[2], hist_range[0]), norm=norm)
axes[1].hist2d(
    Jphi[select], Jz[select], bins=(bins, bins), 
    range=(hist_range[2], hist_range[1]), norm=norm)
axes[2].hist2d(
    Jr[select], Jz[select], bins=(bins, bins), 
    range=(hist_range[0], hist_range[1]), norm=norm)


axes[0].set_xlabel(r' $J_\phi [\mathrm{kpc}^2 \mathrm{Myr}^{-1}]$')
axes[1].set_xlabel(r' $J_\phi [\mathrm{kpc}^2 \mathrm{Myr}^{-1}]$')
axes[2].set_xlabel(r' $J_r [\mathrm{kpc}^2 \mathrm{Myr}^{-1}]$')
axes[0].set_ylabel(r' $J_r [\mathrm{kpc}^2 \mathrm{Myr}^{-1}]$')
axes[1].set_ylabel(r' $J_z [\mathrm{kpc}^2 \mathrm{Myr}^{-1}]$')
axes[2].set_ylabel(r' $J_z [\mathrm{kpc}^2 \mathrm{Myr}^{-1}]$')

fig.tight_layout()
fig.savefig(f'figures/action/subhalo{subhalo_id}_n{i}.png', dpi=300, bbox_inches='tight')